In [0]:
# Design 3 level architecture

bronze_path = "dbfs:/Volumes/workspace/default/kaggle_volume/bronze/events"
silver_path = "dbfs:/Volumes/workspace/default/kaggle_volume/silver/events"
gold_path   = "dbfs:/Volumes/workspace/default/kaggle_volume/gold/daily_sales"


In [0]:
# Build BRONZE Layer (Raw Ingestion)

from pyspark.sql.functions import current_timestamp
# Load raw CSV
raw_events = spark.read.csv("dbfs:/Volumes/workspace/default/kaggle_volume/2019-Oct.csv",
                            header=True,inferSchema=True)

# Add ingestion metadata
bronze_events = raw_events.withColumn("ingestion_time", current_timestamp())

# Write to Bronze Delta
bronze_events.write.format("delta").mode("overwrite").save(bronze_path)

print("Bronze layer created")

Bronze layer created


In [0]:
# Build SILVER Layer (Clean & Validate)

from pyspark.sql import functions as F

# Read Bronze
bronze_df = spark.read.format("delta").load(bronze_path)

# Cleaning
silver_events = (bronze_df
                 .filter(F.col("user_id").isNotNull())
                 .filter(F.col("product_id").isNotNull())
                 .filter(F.col("price") > 0)
                 .dropDuplicates(["user_id", "event_time", "product_id"])
                 )

# Write to Silver Delta
silver_events.write.format("delta").mode("overwrite").save(silver_path)

print("Silver layer created")

Silver layer created


In [0]:
# Build GOLD Layer (Business Aggregates)

# Read Silver
silver_df = spark.read.format("delta").load(silver_path)

# Business metrics
gold_daily_sales = (
    silver_df
    .withColumn("event_date", F.to_date("event_time"))
    .groupBy("event_date", "event_type")
    .agg(
        F.count("*").alias("total_events"),
        F.round(F.sum("price"), 2).alias("total_revenue")
    )
)

# Write to Gold Delta
gold_daily_sales.write.format("delta").mode("overwrite").save(gold_path)

print("Gold layer created")

Gold layer created


In [0]:

display(spark.read.format("delta").load(gold_path))


event_date,event_type,total_events,total_revenue
2019-10-05,view,1269744,3.7581717884E8
2019-10-01,cart,16191,5984602.38
2019-10-09,cart,17886,6145191.18
2019-10-21,view,1342063,3.8172530808E8
2019-10-15,view,1462000,4.3129987233E8
2019-10-15,cart,47786,1.5936543E7
2019-10-12,purchase,25373,7307691.57
2019-10-24,purchase,23999,7470344.6
2019-10-22,purchase,25383,7913671.44
2019-10-29,purchase,20371,6431878.88
